# Permuted Pixel MNIST Demo 
Light weighted demo of our DilatedRNN on Pixel MNist with permutation.

In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from tensorflow.python.client import device_lib
#print device_lib.list_local_devices()

In [2]:
import sys
sys.path.append("./models")
import numpy as np
import tensorflow as tf
from classification_models import drnn_classification
#from pdrnn import drnn_classification

/home/logos/envs/prob_dropout/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# configurations
# processed data from the Zoneout paper:
# https://github.com/teganmaharaj/wtfcptb/raw/master/char_level_penntree.npz
data_path = "./penn_tree_clean.npz"
n_steps = 100 #length of input sequence
input_dims = 128 # char embedding dimension
n_classes = 49 # vocab size

# model config
cell_type = "GRU"
assert(cell_type in ["RNN", "LSTM", "GRU"])
hidden_structs = [256] * 7
dilations = [1,2,4,6,8,32,64]
assert(len(hidden_structs) == len(dilations))

# learning config
batch_size = 64
learning_rate = 0.002
training_iters = batch_size * 30000
testing_step = 10
display_step = 10

test_batch_size = 128


In [4]:
class PTB_data1(object):
    def __init__(self, data_dict, n_steps, val_batch_size):
        self.data_dict = data_dict
        train_size = self.data_dict['train'].shape[0]
        self.train_rng = np.arange(train_size-n_steps-1)
        validation_size = self.data_dict['valid'].shape[0]
        self.n_batches = (validation_size-n_steps-1) / val_batch_size
        self.val_batch_size = val_batch_size
        self.n_steps = n_steps
        
    def random_train_batch(self, batch_size, n_steps):
        """
        Randomly sample a batch of sequences of length n_steps from training set.
        """
        offsets = np.random.choice(self.train_rng, batch_size)
        selections = np.tile(np.arange(n_steps)[:,np.newaxis],
                             batch_size) + offsets
        
        batch_x = self.data_dict['train'][selections]
        batch_y = self.data_dict['train'][offsets+n_steps]
        
        return batch_x.T, batch_y
    
    def get_validation_batches(self):
        """
        A generator for all validation data in mini batches.
        """        
        for batch_id in range(self.n_batches):
            offset_base = np.arange(batch_id*self.val_batch_size, 
                                    (batch_id+1)*self.val_batch_size)
            selections = np.tile(np.arange(self.n_steps)[:,np.newaxis],
                                 self.val_batch_size) + offset_base
            batch_x = self.data_dict['valid'][selections]
            batch_y = self.data_dict['valid'][offset_base+self.n_steps]
            yield batch_x.T, batch_y

data_path = "./penn_tree_clean.npz"            
ptb_data1 = PTB_data1(np.load(data_path), 100, 16384)

In [46]:
a = np.load(data_path)
b = a['valid']
b.shape

(396412,)

In [42]:
offset_base = np.arange(2*batch_size, (2+1)*batch_size)
selections = np.tile(np.arange(n_steps)[:,np.newaxis],
                     batch_size) + offset_base

In [45]:
selections

array([[128, 129, 130, ..., 189, 190, 191],
       [129, 130, 131, ..., 190, 191, 192],
       [130, 131, 132, ..., 191, 192, 193],
       ...,
       [225, 226, 227, ..., 286, 287, 288],
       [226, 227, 228, ..., 287, 288, 289],
       [227, 228, 229, ..., 288, 289, 290]])

In [53]:
class PTB_data2(object):
    def __init__(self, data_dict):
        self.data_dict = data_dict
        
    def random_train_batch(self, batch_size, n_steps):
        """
        Randomly sample a batch of sequences of length n_steps from training set.
        """
        np.random.seed(0)
        batch_x = np.zeros((batch_size, n_steps), dtype=np.int32)
        batch_y = np.zeros((batch_size,), dtype=np.int32)
        train_size = len(self.data_dict['train'])
        offsets = np.random.choice(range(train_size-n_steps-1), batch_size)
        for idx in range(batch_size):
            offset = offsets[idx]
            batch_x[idx,:] = self.data_dict['train'][offset:offset+n_steps]
            batch_y[idx] = self.data_dict['train'][offset+n_steps]
        return batch_x, batch_y
    
    def get_validation_batches(self, batch_size, n_steps):
        """
        A generator for all validation data in mini batches.
        """
        validation_size = len(self.data_dict['valid'])
        batch_x = np.zeros((batch_size, n_steps), dtype=np.int32)
        batch_y = np.zeros((batch_size,), dtype=np.int32)
        n_batches = (validation_size-n_steps-1) / batch_size
        
        for batch_id in range(n_batches):
            offset_base = batch_id * batch_size
            for idx in range(batch_size):
                offset = offset_base + idx
                batch_x[idx,:] = self.data_dict['valid'][offset:offset+n_steps]
                batch_y[idx] = self.data_dict['valid'][offset+n_steps]
            yield batch_x.T, batch_y[np.newaxis,:]

ptb_data2 = PTB_data2(np.load(data_path))

In [51]:
inputs_val_1, target_val_1 = np.zeros((100,128)), np.zeros((1,128)) 
for inputs_val, target_val in ptb_data1.get_validation_batches():
    inputs_val_1 += inputs_val
    target_val_1 += target_val

In [54]:
inputs_val_2, target_val_2 = np.zeros((100,128)), np.zeros((1,128)) 
for inputs_val, target_val in ptb_data2.get_validation_batches(128, 100):
    inputs_val_2 += inputs_val
    target_val_2 += target_val

In [62]:
target_val_2

array([[21972., 21993., 22038., 22657., 22465., 21922., 21545., 22376.,
        21668., 22192., 21421., 22226., 22148., 21401., 22260., 21772.,
        21698., 21941., 22006., 21882., 22729., 21534., 21983., 22023.,
        22279., 22825., 21668., 22769., 21601., 22906., 21908., 22115.,
        22000., 22127., 22027., 22164., 22490., 22198., 21570., 22495.,
        21679., 22697., 22397., 22054., 22856., 21291., 22394., 22383.,
        21456., 22827., 21356., 22243., 21358., 22055., 22554., 23019.,
        21842., 22377., 22054., 21736., 22070., 22383., 21996., 22147.,
        22081., 22459., 20684., 23103., 22056., 21585., 22107., 21526.,
        21885., 22090., 21748., 21737., 22811., 22187., 21491., 22659.,
        22384., 20828., 22342., 21646., 21563., 21847., 21957., 21816.,
        22127., 21542., 21472., 21846., 22001., 22129., 21368., 21923.,
        22055., 22430., 22454., 22168., 21905., 22938., 22010., 22390.,
        21943., 21977., 22391., 22229., 21404., 22611., 21811., 

In [15]:
#char_embeddings = tf.get_variable("char_embeddings", [n_classes, input_dims])
x_emb2 = tf.nn.embedding_lookup(char_embeddings, batch_x)

In [38]:
x_emb

<tf.Tensor 'embedding_lookup:0' shape=(64, 100, 64) dtype=float32>

In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(tf.reduce_all(tf.equal(x_emb1, tf.transpose(x_emb2, perm=[1,0,2]))))

True


In [6]:
# build computation graph
tf.reset_default_graph()
x = tf.placeholder(tf.int32, [None, n_steps])
y = tf.placeholder(tf.int32, [None,])

char_embeddings = tf.get_variable("char_embeddings", [n_classes, input_dims])
x_emb = tf.nn.embedding_lookup(char_embeddings, x)

global_step = tf.Variable(0, name='global_step', trainable=False)

# build prediction graph
print "==> Building a dRNN with %s cells" %cell_type
#pred = drnn_classification(x_emb, hidden_structs, dilations, n_classes, 1, False, input_dims, cell_type)
pred = drnn_classification(x_emb, hidden_structs, dilations, n_steps, n_classes, input_dims, cell_type)

# build loss and optimizer
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=y))
bpc_cost = cost / np.log(2.0)
optimizer = tf.train.AdamOptimizer(learning_rate)
gvs = optimizer.compute_gradients(bpc_cost)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

==> Building a dRNN with GRU cells
Building layer: multi_dRNN_dilation_1, input length: 100, dilation rate: 1, input dim: 128.
=====> Input length for sub-RNN: 100
Building layer: multi_dRNN_dilation_2, input length: 100, dilation rate: 2, input dim: 256.
=====> Input length for sub-RNN: 50
Building layer: multi_dRNN_dilation_4, input length: 100, dilation rate: 4, input dim: 256.
=====> Input length for sub-RNN: 25
Building layer: multi_dRNN_dilation_6, input length: 100, dilation rate: 6, input dim: 256.
=====> 2 time points need to be padded. 
=====> Input length for sub-RNN: 17
Building layer: multi_dRNN_dilation_8, input length: 100, dilation rate: 8, input dim: 256.
=====> 4 time points need to be padded. 
=====> Input length for sub-RNN: 13
Building layer: multi_dRNN_dilation_32, input length: 100, dilation rate: 32, input dim: 256.
=====> 28 time points need to be padded. 
=====> Input length for sub-RNN: 4
Building layer: multi_dRNN_dilation_64, input length: 100, dilation rat

In [9]:
gvs

[(<tensorflow.python.framework.ops.IndexedSlices at 0x7f6219269c50>,
  <tf.Variable 'char_embeddings:0' shape=(49, 128) dtype=float32_ref>),
 (<tf.Tensor 'gradients/AddN_662:0' shape=(384, 512) dtype=float32>,
  <tf.Variable 'multi_dRNN_dilation_1/gru_cell/gates/kernel:0' shape=(384, 512) dtype=float32_ref>),
 (<tf.Tensor 'gradients/AddN_661:0' shape=(512,) dtype=float32>,
  <tf.Variable 'multi_dRNN_dilation_1/gru_cell/gates/bias:0' shape=(512,) dtype=float32_ref>),
 (<tf.Tensor 'gradients/AddN_659:0' shape=(384, 256) dtype=float32>,
  <tf.Variable 'multi_dRNN_dilation_1/gru_cell/candidate/kernel:0' shape=(384, 256) dtype=float32_ref>),
 (<tf.Tensor 'gradients/AddN_658:0' shape=(256,) dtype=float32>,
  <tf.Variable 'multi_dRNN_dilation_1/gru_cell/candidate/bias:0' shape=(256,) dtype=float32_ref>),
 (<tf.Tensor 'gradients/AddN_503:0' shape=(512, 512) dtype=float32>,
  <tf.Variable 'multi_dRNN_dilation_2/gru_cell/gates/kernel:0' shape=(512, 512) dtype=float32_ref>),
 (<tf.Tensor 'gradien

In [7]:
tf.trainable_variables()

[<tf.Variable 'char_embeddings:0' shape=(49, 64) dtype=float32_ref>,
 <tf.Variable 'multi_dRNN_dilation_1/basic_lstm_cell/kernel:0' shape=(320, 1024) dtype=float32_ref>,
 <tf.Variable 'multi_dRNN_dilation_1/basic_lstm_cell/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Variable:0' shape=(256, 49) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(49,) dtype=float32_ref>]

In [6]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [7]:
step = 0
train_results = []
validation_results = []
test_results = []

while step < 10000000:
    batch_x, batch_y = ptb_data1.random_train_batch(batch_size, n_steps)    

    feed_dict = {
        x : batch_x,
        y : batch_y
    }
    bpc_cost_, step_,  _ = sess.run([bpc_cost, global_step, train_op], feed_dict=feed_dict)    
    #train_results.append((step_, bpc_cost_))    

    if (step + 1) % 10 == 0:
        print "Iter " + str(step + 1) + ", Minibatch Loss: " + "{:.6f}".format(bpc_cost_)
             
    if (step + 1) % 300 == 0:
        
        # validation performance
        batch_bpcs = []
        for batch_x, batch_y in ptb_data1.get_validation_batches():
            feed_dict = {
                x : batch_x,
                y : batch_y
            }
            bpc_cost_, step_ = sess.run([bpc_cost, global_step], feed_dict=feed_dict) 
            batch_bpcs.append(bpc_cost_)
        validation_bpc = np.mean(batch_bpcs)
        print "========> Validation BPC: " + "{:.6f}".format(validation_bpc) + " over %d batches" % len(batch_bpcs)
    step += 1

Iter 10, Minibatch Loss: 4.672160
Iter 20, Minibatch Loss: 4.374793
Iter 30, Minibatch Loss: 4.621165
Iter 40, Minibatch Loss: 4.525662
Iter 50, Minibatch Loss: 4.514990
Iter 60, Minibatch Loss: 4.323358
Iter 70, Minibatch Loss: 4.107100
Iter 80, Minibatch Loss: 4.269569
Iter 90, Minibatch Loss: 4.314702
Iter 100, Minibatch Loss: 4.593975
Iter 110, Minibatch Loss: 4.401452
Iter 120, Minibatch Loss: 4.606582
Iter 130, Minibatch Loss: 3.982379
Iter 140, Minibatch Loss: 4.511949
Iter 150, Minibatch Loss: 4.164510
Iter 160, Minibatch Loss: 4.541777
Iter 170, Minibatch Loss: 4.232868
Iter 180, Minibatch Loss: 4.600549
Iter 190, Minibatch Loss: 4.343833
Iter 200, Minibatch Loss: 4.520821
Iter 210, Minibatch Loss: 4.168346
Iter 220, Minibatch Loss: 3.921391
Iter 230, Minibatch Loss: 4.508394
Iter 240, Minibatch Loss: 4.462979
Iter 250, Minibatch Loss: 4.182891
Iter 260, Minibatch Loss: 4.166235
Iter 270, Minibatch Loss: 4.409044
Iter 280, Minibatch Loss: 4.683818
Iter 290, Minibatch Loss: 4.4

ResourceExhaustedError: OOM when allocating tensor with shape[524288,512]
	 [[Node: multi_dRNN_dilation_32/multi_dRNN_dilation_32/gru_cell/concat_1 = ConcatV2[N=2, T=DT_DOUBLE, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](concat_205, multi_dRNN_dilation_32/multi_dRNN_dilation_32/gru_cell/mul, multi_dRNN_dilation_64/multi_dRNN_dilation_64/gru_cell/split_1/split_dim)]]
	 [[Node: div/_37 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7286_div", tensor_type=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'multi_dRNN_dilation_32/multi_dRNN_dilation_32/gru_cell/concat_1', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals

In [40]:
logits = sess.run(pred, feed_dict=feed_dict)
correct_pred = np.equal(np.argmax(logits,-1), batch_y)
print np.mean(correct_pred.astype(np.float32))

0.5625
